In [1]:
import os
import glob
import hashlib
from pathlib import Path
from typing import List
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage
from langchain.tools import tool
from langgraph.checkpoint.memory import MemorySaver 

load_dotenv()

PDF_DIR = os.getenv("PDF_DIR")

COLLECTION_NAME=os.getenv("COLLECTION_NAME")

OLLAMA_MODEL=os.getenv("OLLAMA_MODEL")

PERSIST_DIR=os.getenv("PERSIST_DIR")

In [2]:
def stable_id(text: str, meta_key: str) -> str:
    """Create a stable, de-duplicated ID for each chunk."""
    h = hashlib.sha256()

    h.update((text + "|" + meta_key).encode("utf-8"))

    return h.hexdigest()

def load_pdfs(pdf_dir: str) -> List[Document]:
    docs: List[Document] = []

    for pdf_path in glob.glob(os.path.join(pdf_dir, "**/*.pdf"), recursive=True):
        loader = PyPDFLoader(pdf_path)
       
        page_docs = loader.load()
        
        for d in page_docs:
            d.metadata.setdefault("source", pdf_path)

        docs.extend(page_docs)

    return docs

def chunk_docs(docs: List[Document]) -> List[Document]:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,      
        chunk_overlap=120,
        separators=["\n\n", "\n", " ", ""],
    )

    return splitter.split_documents(docs)

print("Creating / updating Chroma collection...")

vectordb = Chroma(
    collection_name=COLLECTION_NAME,
    embedding_function=OllamaEmbeddings(model=OLLAMA_MODEL),
    persist_directory=PERSIST_DIR,
)

Creating / updating Chroma collection...


In [3]:
def load_and_save_embeddings_to_db():
    Path(PERSIST_DIR).mkdir(parents=True, exist_ok=True)

    print("Loading PDFs...")

    raw_docs = load_pdfs(PDF_DIR)

    if not raw_docs:
        print(f"No PDFs found in the directory.")
        return

    print(f"Loaded {len(raw_docs)} page docs. Chunking...")

    docs = chunk_docs(raw_docs)

    print(f"Produced {len(docs)} chunks.")

    # Avoid duplicating chunks on re-runs by providing stable IDs
    ids = []

    for d in docs:
        # include source + page + start offset to reduce collisions
        mk = f"{d.metadata.get('source','')}-{d.metadata.get('page',-1)}-{len(d.page_content)}"
        
        ids.append(stable_id(d.page_content, mk))

    # Upsert into Chroma
    vectordb.add_documents(documents=docs, ids=ids)

    print("documents added to the collection")

# Loading docs and save embeddings to the DB

In [4]:
load_and_save_embeddings_to_db()

Loading PDFs...


Ignoring wrong pointing object 151 0 (offset 0)
Ignoring wrong pointing object 286 0 (offset 0)
Ignoring wrong pointing object 440 0 (offset 0)
Ignoring wrong pointing object 502 0 (offset 0)
Ignoring wrong pointing object 523 0 (offset 0)
Ignoring wrong pointing object 555 0 (offset 0)
Ignoring wrong pointing object 558 0 (offset 0)
Ignoring wrong pointing object 566 0 (offset 0)
Ignoring wrong pointing object 587 0 (offset 0)


Loaded 47 page docs. Chunking...
Produced 90 chunks.
documents added to the collection


In [5]:
# retreiver config
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

In [6]:
@tool
def retrieve_context(query:str):
    """Search for the info related ot the given query in the vector database"""
    
    try:
        docs = retriever.invoke(query)

        # print(f"\nTop {len(docs)} chunks:")

        # for i, doc in enumerate(docs):
        #     src = doc.metadata.get("source", "unknown")
        #     page = doc.metadata.get("page", "?")
        #     print(f"\n[{i}] {src} (page {page})\n{doc.page_content[:500]}...")

        content = "\n".join([doc.page_content for doc in docs])

        return content
    except Exception as e:
        print(f"Error in retrieving the information:{e}")

        return f"Error in retrieving the information {e}. Please try again."


# Creating the agent with RAG

In [11]:
llm_model = ChatOllama(model="llama3.2", temperature=0.4, repeat_penalty=1.3)

agent_executor = create_agent(model=llm_model, 
                            tools=[retrieve_context], 
                            checkpointer=MemorySaver(),
                            system_prompt = """
                                You are a **knowledge-check assistant** for the Keerti Gen-AI course with random concepts.

                                STRICT RULES:
                                1. You may use ONLY the `retrieve_context` tool — and ONLY when you need background info
                                before writing a question. Never use it after a user answers.
                                2. When grading, DO NOT call any tool or emit JSON / code / <|python_tag|>. 
                                Simply say whether the answer is correct or wrong and add a one-sentence explanation.
                                3. Always generate ONE multiple-choice question (A–D) at a time.
                                4. Wait for the user's answer, then evaluate it.
                                5. After evaluating, pick another concept, call `retrieve_context` with that concept,
                                and generate the next question.
                                6. Use plain English text only — no JSON, no special tags.
                                """
                            )

In [12]:
def format_last_ai(response):
    response["messages"][-1].pretty_print()

In [13]:
def start_quiz():
    print("Keerti Gen-AI Quiz • type 'exit' to quit")
    print("Assistant: Ready! I’ll pick a concept and generate the first question.")

    config = {"configurable": {"thread_id": "keerti-genai-quiz"}}

    first_turn = agent_executor.invoke(
                        {"messages": [HumanMessage(content="Start the quiz.")]}, 
                        config=config)
    
    format_last_ai(first_turn)

    while True:
        user_input = input("Your option: ").strip()

        if user_input.lower() in {"exit", "quit"}:
            print("Assistant: Great session! See you next time.")
            break
        
        print(user_input)

        user_message = HumanMessage(content=f"My answer is: {user_input}. When grading, DO NOT call any tool or emit JSON / code / <|python_tag|>. Simply say whether the answer is correct or wrong and add a one-sentence explanation. Generate next question with 4 options.")

        # user_message.pretty_print()

        turn = agent_executor.invoke(
                {"messages": [user_message]}, 
                config=config)

        format_last_ai(turn)

In [14]:
start_quiz()

Keerti Gen-AI Quiz • type 'exit' to quit
Assistant: Ready! I’ll pick a concept and generate the first question.
================================== Ai Message ==================================

Let's start the quiz.

Here is your first question:

What is a key purpose of callbacks in Keras?

A) To train models with more data
B) To enhance control over the training process and automate tasks like saving the best model or adjusting learning rates.
C) To improve model performance by using different optimization algorithms
D) To reduce the number of parameters in neural networks

Please choose your answer.
D
================================== Ai Message ==================================

Wrong
The best answer is B. A callback's primary purpose in Keras is to enhance control over the training process, allowing for tasks like saving the best model or adjusting learning rates.

Here's your next question:

What type of data does a Generative Adversarial Network (GAN) generate?

A) Realistic i